In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
import numpy as np
import random

In [2]:
ctx = utils.try_all_gpus()
#mlp = MLP()
lenet = CNN_Model('LeNet')


def get_data(data_idx, batch_size):
    data = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data_random\\train_data" + str(data_idx) + ".npy"
    label = "E:\\PythonProjects\\Mxnet_FederatedLearning\\Fed_Client\\FedAvg_data_random\\train_label" + str(data_idx) + ".npy"
    data = np.load(data)
    label = np.load(label)
    train_data = mx.io.NDArrayIter(data,label,batch_size=batch_size,shuffle=True)
    return train_data

def validate(net):
    mnist = mx.test_utils.get_mnist()
    val_data = mx.io.NDArrayIter(mnist['test_data'],mnist['test_label'],batch_size=100)
    metric = mx.metric.Accuracy()
    val_data.reset()
    for batch in val_data:
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    name,acc =metric.get()
    return acc

In [3]:
def train_till_acc(net, stop_acc, learning_rate, batch_size, epoch):
    round = 0
    clist = [x for x in range(100)]
    acc_rec = []
    while True:
        random.shuffle(clist)
        for client in clist:
            train_data = get_data(client, batch_size)
            softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
            trainer = trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})
            metric = mx.metric.Accuracy()
            stop_flag = False
            print("Round %d starting"%(round))
            for i in range(epoch):
                train_data.reset()
                for batch in train_data:
                    data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
                    label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
                    outputs = []
                    with ag.record():
                        for x, y in zip(data, label):
                            z = net(x)
                            loss = softmax_cross_entropy_loss(z, y)
                            loss.backward()
                            outputs.append(z)
                    metric.update(label, outputs)
                    trainer.step(batch.data[0].shape[0])
                name, acc = metric.get()
                metric.reset()
                print('training acc at epoch %d: %s=%f'%(i, name, acc))
            acc = validate(net)
            print("accuracy in test set %f"%(acc))
            acc_rec.append(acc)
            if acc >= stop_acc:
                stop_flag = True
                break
            round += 1
        if stop_flag:
            print("round: ",round)
            path =  "E:\\PythonProjects\\Mxnet_FederatedLearning\\exp_rec\\" + str(epoch) + "-" + str(batch_size) + "-niid"
            file_handle = open(path,"w")
            file_handle.write(str(acc_rec))
            break

In [4]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,600,1)

26 starting
training acc at epoch 0: accuracy=0.966667
accuracy in test set 0.976200
Round 2727 starting
training acc at epoch 0: accuracy=0.983333
accuracy in test set 0.975800
Round 2728 starting
training acc at epoch 0: accuracy=0.985000
accuracy in test set 0.976900
Round 2729 starting
training acc at epoch 0: accuracy=0.968333
accuracy in test set 0.977900
Round 2730 starting
training acc at epoch 0: accuracy=0.978333
accuracy in test set 0.976800
Round 2731 starting
training acc at epoch 0: accuracy=0.981667
accuracy in test set 0.978700
Round 2732 starting
training acc at epoch 0: accuracy=0.983333
accuracy in test set 0.977200
Round 2733 starting
training acc at epoch 0: accuracy=0.971667
accuracy in test set 0.977600
Round 2734 starting
training acc at epoch 0: accuracy=0.978333
accuracy in test set 0.977300
Round 2735 starting
training acc at epoch 0: accuracy=0.968333
accuracy in test set 0.978100
Round 2736 starting
training acc at epoch 0: accuracy=0.973333
accuracy in tes

In [5]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,600,5)

t epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.991667
training acc at epoch 3: accuracy=0.993333
training acc at epoch 4: accuracy=0.993333
accuracy in test set 0.976200
Round 675 starting
training acc at epoch 0: accuracy=0.973333
training acc at epoch 1: accuracy=0.975000
training acc at epoch 2: accuracy=0.978333
training acc at epoch 3: accuracy=0.981667
training acc at epoch 4: accuracy=0.985000
accuracy in test set 0.979200
Round 676 starting
training acc at epoch 0: accuracy=0.973333
training acc at epoch 1: accuracy=0.978333
training acc at epoch 2: accuracy=0.981667
training acc at epoch 3: accuracy=0.981667
training acc at epoch 4: accuracy=0.981667
accuracy in test set 0.979300
Round 677 starting
training acc at epoch 0: accuracy=0.970000
training acc at epoch 1: accuracy=0.978333
training acc at epoch 2: accuracy=0.981667
training acc at epoch 3: accuracy=0.981667
training acc at epoch 4: accuracy=0.985000
accuracy in test set 0.979100
Round 678 starting
t

In [6]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,50,1)

cy in test set 0.950800
Round 102 starting
training acc at epoch 0: accuracy=0.950000
accuracy in test set 0.952200
Round 103 starting
training acc at epoch 0: accuracy=0.943333
accuracy in test set 0.956900
Round 104 starting
training acc at epoch 0: accuracy=0.955000
accuracy in test set 0.960900
Round 105 starting
training acc at epoch 0: accuracy=0.955000
accuracy in test set 0.960900
Round 106 starting
training acc at epoch 0: accuracy=0.961667
accuracy in test set 0.951300
Round 107 starting
training acc at epoch 0: accuracy=0.966667
accuracy in test set 0.959200
Round 108 starting
training acc at epoch 0: accuracy=0.953333
accuracy in test set 0.954800
Round 109 starting
training acc at epoch 0: accuracy=0.960000
accuracy in test set 0.962400
Round 110 starting
training acc at epoch 0: accuracy=0.958333
accuracy in test set 0.955100
Round 111 starting
training acc at epoch 0: accuracy=0.971667
accuracy in test set 0.961400
Round 112 starting
training acc at epoch 0: accuracy=0.9

In [7]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,600,20)

333
training acc at epoch 5: accuracy=0.993333
training acc at epoch 6: accuracy=0.993333
training acc at epoch 7: accuracy=0.993333
training acc at epoch 8: accuracy=0.993333
training acc at epoch 9: accuracy=0.995000
training acc at epoch 10: accuracy=0.995000
training acc at epoch 11: accuracy=0.996667
training acc at epoch 12: accuracy=0.996667
training acc at epoch 13: accuracy=0.998333
training acc at epoch 14: accuracy=0.998333
training acc at epoch 15: accuracy=0.998333
training acc at epoch 16: accuracy=0.998333
training acc at epoch 17: accuracy=0.998333
training acc at epoch 18: accuracy=0.998333
training acc at epoch 19: accuracy=0.998333
accuracy in test set 0.978600
Round 247 starting
training acc at epoch 0: accuracy=0.968333
training acc at epoch 1: accuracy=0.965000
training acc at epoch 2: accuracy=0.971667
training acc at epoch 3: accuracy=0.970000
training acc at epoch 4: accuracy=0.976667
training acc at epoch 5: accuracy=0.986667
training acc at epoch 6: accuracy=

In [8]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,10,1)

Round 0 starting
training acc at epoch 0: accuracy=0.160000
accuracy in test set 0.219500
Round 1 starting
training acc at epoch 0: accuracy=0.425000
accuracy in test set 0.673900
Round 2 starting
training acc at epoch 0: accuracy=0.655000
accuracy in test set 0.725300
Round 3 starting
training acc at epoch 0: accuracy=0.751667
accuracy in test set 0.860700
Round 4 starting
training acc at epoch 0: accuracy=0.841667
accuracy in test set 0.874300
Round 5 starting
training acc at epoch 0: accuracy=0.858333
accuracy in test set 0.896800
Round 6 starting
training acc at epoch 0: accuracy=0.875000
accuracy in test set 0.900100
Round 7 starting
training acc at epoch 0: accuracy=0.920000
accuracy in test set 0.917000
Round 8 starting
training acc at epoch 0: accuracy=0.901667
accuracy in test set 0.860800
Round 9 starting
training acc at epoch 0: accuracy=0.886667
accuracy in test set 0.910700
Round 10 starting
training acc at epoch 0: accuracy=0.920000
accuracy in test set 0.908100
Round 11 

In [9]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,50,5)

ound 25 starting
training acc at epoch 0: accuracy=0.955000
training acc at epoch 1: accuracy=0.963333
training acc at epoch 2: accuracy=0.983333
training acc at epoch 3: accuracy=0.983333
training acc at epoch 4: accuracy=0.988333
accuracy in test set 0.956200
Round 26 starting
training acc at epoch 0: accuracy=0.948333
training acc at epoch 1: accuracy=0.965000
training acc at epoch 2: accuracy=0.973333
training acc at epoch 3: accuracy=0.971667
training acc at epoch 4: accuracy=0.983333
accuracy in test set 0.956100
Round 27 starting
training acc at epoch 0: accuracy=0.920000
training acc at epoch 1: accuracy=0.955000
training acc at epoch 2: accuracy=0.961667
training acc at epoch 3: accuracy=0.963333
training acc at epoch 4: accuracy=0.971667
accuracy in test set 0.955300
Round 28 starting
training acc at epoch 0: accuracy=0.943333
training acc at epoch 1: accuracy=0.965000
training acc at epoch 2: accuracy=0.978333
training acc at epoch 3: accuracy=0.981667
training acc at epoch 

In [10]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,50,20)

poch 4: accuracy=1.000000
training acc at epoch 5: accuracy=1.000000
training acc at epoch 6: accuracy=1.000000
training acc at epoch 7: accuracy=1.000000
training acc at epoch 8: accuracy=1.000000
training acc at epoch 9: accuracy=1.000000
training acc at epoch 10: accuracy=1.000000
training acc at epoch 11: accuracy=1.000000
training acc at epoch 12: accuracy=1.000000
training acc at epoch 13: accuracy=1.000000
training acc at epoch 14: accuracy=1.000000
training acc at epoch 15: accuracy=1.000000
training acc at epoch 16: accuracy=1.000000
training acc at epoch 17: accuracy=1.000000
training acc at epoch 18: accuracy=1.000000
training acc at epoch 19: accuracy=1.000000
accuracy in test set 0.979000
Round 74 starting
training acc at epoch 0: accuracy=0.953333
training acc at epoch 1: accuracy=0.976667
training acc at epoch 2: accuracy=0.983333
training acc at epoch 3: accuracy=0.995000
training acc at epoch 4: accuracy=1.000000
training acc at epoch 5: accuracy=1.000000
training acc 

In [11]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,10,5)

Round 0 starting
training acc at epoch 0: accuracy=0.180000
training acc at epoch 1: accuracy=0.531667
training acc at epoch 2: accuracy=0.758333
training acc at epoch 3: accuracy=0.863333
training acc at epoch 4: accuracy=0.893333
accuracy in test set 0.837200
Round 1 starting
training acc at epoch 0: accuracy=0.865000
training acc at epoch 1: accuracy=0.900000
training acc at epoch 2: accuracy=0.908333
training acc at epoch 3: accuracy=0.925000
training acc at epoch 4: accuracy=0.943333
accuracy in test set 0.902500
Round 2 starting
training acc at epoch 0: accuracy=0.886667
training acc at epoch 1: accuracy=0.918333
training acc at epoch 2: accuracy=0.948333
training acc at epoch 3: accuracy=0.960000
training acc at epoch 4: accuracy=0.968333
accuracy in test set 0.918100
Round 3 starting
training acc at epoch 0: accuracy=0.921667
training acc at epoch 1: accuracy=0.953333
training acc at epoch 2: accuracy=0.970000
training acc at epoch 3: accuracy=0.971667
training acc at epoch 4: 

In [12]:
lenet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(lenet,0.98,0.02,10,20)

poch 4: accuracy=1.000000
training acc at epoch 5: accuracy=1.000000
training acc at epoch 6: accuracy=1.000000
training acc at epoch 7: accuracy=1.000000
training acc at epoch 8: accuracy=1.000000
training acc at epoch 9: accuracy=1.000000
training acc at epoch 10: accuracy=1.000000
training acc at epoch 11: accuracy=1.000000
training acc at epoch 12: accuracy=1.000000
training acc at epoch 13: accuracy=1.000000
training acc at epoch 14: accuracy=1.000000
training acc at epoch 15: accuracy=1.000000
training acc at epoch 16: accuracy=1.000000
training acc at epoch 17: accuracy=1.000000
training acc at epoch 18: accuracy=1.000000
training acc at epoch 19: accuracy=1.000000
accuracy in test set 0.975500
Round 30 starting
training acc at epoch 0: accuracy=0.965000
training acc at epoch 1: accuracy=0.988333
training acc at epoch 2: accuracy=0.993333
training acc at epoch 3: accuracy=1.000000
training acc at epoch 4: accuracy=1.000000
training acc at epoch 5: accuracy=1.000000
training acc 